# Save ALS point clouds from Estonian Land Board 

This notebook is for downloading point clouds from Estonian Land Board in .laz format and saving them to external harddrive. The point clouds are downloaded according to a .csv table that stores the map square number and the year of the point cloud needed. Map squares are 1:2000 map squares covering Estonia and how the national ALS data is structured. They can be downloaded from ELB website as a vector/feature layer. The map squares to be downloaded were selected in QGIS according to which have overlap with ICESat-2 transects with forest in them.

The steps that were taken for retrieving the .csv file:
* Add to QGIS: Hansen intact forest areas, tree species map, map squares, icesat transects' polygons
* Select only the ICESat transects that are inside intact forest
* Select map squares that have forested icesat transects in them using Extract by location tool.
* Export the selected map squares as .csv

In [1]:
# import the necessary libraries
import os
from urllib import request
from urllib import error

import pandas as pd
import geopandas as gp

In [2]:
# read the .csv into a pandas dataframe
als_list = pd.read_csv('..\\Data\\ALS_files_list.csv')
als_list[als_list['ALS_METS'].notnull()]

,NR,NR10000,ALS_AASTAD,ALS_TAVA_1,ALS_TAVA_2,ALS_TAVA_3,ALS_TAVA_4,ALS_MADAL,ALS_METS
75,377650,44753,"2019,2015,2011",2011.0,2015.0,2019.0,NaN,NaN,2017.0
76,377651,44753,"2019,2015,2011",2011.0,2015.0,2019.0,NaN,NaN,2017.0
77,377652,44753,"2019,2015,2011",2011.0,2015.0,2019.0,NaN,NaN,2017.0
78,377653,44753,"2019,2015,2011",2011.0,2015.0,2019.0,NaN,NaN,2017.0
80,377655,44754,"2019,2015,2011",2011.0,2015.0,2019.0,NaN,NaN,2017.0
...,...,...,...,...,...,...,...,...,...
57861,616601,74103,"2022,2018,2013,2009",2009.0,2013.0,2018.0,2022.0,NaN,2020.0
57862,616602,74103,"2022,2018,2013,2009",2009.0,2013.0,2018.0,2022.0,NaN,2020.0
57913,617600,74103,"2022,2018,2013,2009",2009.0,2013.0,2018.0,2022.0,NaN,2020.0
57914,617601,74103,"2022,2018,2013,2009",2009.0,2013.0,2018.0,2022.0,NaN,2020.0


In [3]:
# Now create a dataframe that has no duplicates
# Also remove entires, where ALS_TAVA_3 or ALS_METS is not existing as these are mainly on the border of the country and 
# data is incomplete
als_list = als_list[als_list['ALS_TAVA_3'].notnull()]
als_list = als_list[als_list['ALS_METS'].notnull()]
als_list = als_list.drop_duplicates(subset = ['NR'])
als_list

,NR,NR10000,ALS_AASTAD,ALS_TAVA_1,ALS_TAVA_2,ALS_TAVA_3,ALS_TAVA_4,ALS_MADAL,ALS_METS
75,377650,44753,"2019,2015,2011",2011.0,2015.0,2019.0,NaN,NaN,2017.0
76,377651,44753,"2019,2015,2011",2011.0,2015.0,2019.0,NaN,NaN,2017.0
77,377652,44753,"2019,2015,2011",2011.0,2015.0,2019.0,NaN,NaN,2017.0
78,377653,44753,"2019,2015,2011",2011.0,2015.0,2019.0,NaN,NaN,2017.0
80,377655,44754,"2019,2015,2011",2011.0,2015.0,2019.0,NaN,NaN,2017.0
...,...,...,...,...,...,...,...,...,...
57861,616601,74103,"2022,2018,2013,2009",2009.0,2013.0,2018.0,2022.0,NaN,2020.0
57862,616602,74103,"2022,2018,2013,2009",2009.0,2013.0,2018.0,2022.0,NaN,2020.0
57913,617600,74103,"2022,2018,2013,2009",2009.0,2013.0,2018.0,2022.0,NaN,2020.0
57914,617601,74103,"2022,2018,2013,2009",2009.0,2013.0,2018.0,2022.0,NaN,2020.0


### Note on the point clouds accessible on Estonian Land Board website
Each year Estonian Land Board collects three types of ALS data:
1. TAVA - regular aerial laser scanning of quarter of Estonia in spring (no-leaf) at 2000 or 2600 m with 2.1 p/m2.
2. METS - aerial laser scanning of quarter of Estonia in the summer (leaf-on) at 3100 m and 0.8 p/m2.
3. MADAL - annual aerial laser scanning of Estonian cities at 1200 m and 18 p/m2 (sometimes 3,5 p/m2). Note - some of these point clouds also include some forested areas.

This notebook iterates over point cloud id-s marking 1:2000 map squares and sees if spring data for this area exists from the ALS 4th round taking place 2021-2024. If not, it downloads the data from the 3rd round that took place 2017-2020. It also downloads the newest point cloud from summer laser scanning.

### Using the spring and summer scans
Tava/regular point clouds have much higer point density - 3.5 p/m. However, they are acquired in spring when there are no leaves on the trees. Therefore, if I were to use only this type, the comparison of canopy gap retrieved from ICESat-2 transect showing leaf-on would differ.

I have decided to download point clouds both acquired in spring and in the summer and test the canopy gap estimates from ICESat-2 data acquired during the respective season. This means, ICESat-2 data acquired between May and September are compared to data from point clouds acquired in the summer, while ICESat-2 transects acquired between October and April are compared to ALS data acquired in spring.

### If and how to use the point clouds of urban areas
Every year a much denser point clouds are made for urban areas, which however also include some forest. Currently, I have 1463 map squares which have forest and one or more ICESat-2 transects. This would allow test the methodology with much higer point density. This is not currently implemented.

### The following function does:
1. Iterate over dataframe row by row
2. Saves the NR field as variable to keep the map square identifier
3. Checks if field ALS_TAVA is NULL. If no, the tava_year is taken from field ALS_TAVA_4. Otherwise, the tava_year is taken from ALS_TAVA_4.
3. Creates the file names for the laz files to be saved both for 'tava' and 'mets' files such as: map_square_nr_year_type.laz
4. Creates the url for downloading the laz file
5. Creates an empty .laz file for each type on the external harddrive
6. Writes the .laz file retrieved from the Estonian Land Board to the empty file.




In [6]:
# folder to store saved laz files
data_loc = 'Z:\\Thesis\\Data\\ALS_data\\'
df = als_list
for iindex, row in df[:10].iterrows():
    
    tava_year = 0
    if pd.isna(row['ALS_METS']):
        # arbitrary year - will not find file with this year anyway
        mets_year = 0
    else:
        mets_year = int(row['ALS_METS'])
    
    map_sq = row['NR']
    
    
    if pd.isna(row['ALS_TAVA_4']):
        # year is ALS_TAVA_3
        tava_year = int(row['ALS_TAVA_3'])
    else:
        # year is ALS_TAVA_4
        tava_year = int(row['ALS_TAVA_4'])
        
        
    tava_laz = str(map_sq) + "_" + str(tava_year) + '_tava.laz'
    mets_laz = str(map_sq) + "_" + str(mets_year) + '_mets.laz'
    
    # create empty files to external hardrive
    tava_file = data_loc + 'TAVA\\' + tava_laz
    mets_file = data_loc + 'METS\\' + mets_laz
    
    
    URL_TAVA = 'https://geoportaal.maaamet.ee/index.php?lang_id=1&plugin_act=otsing&kaardiruut=' + str(map_sq) + '&andmetyyp=lidar_laz_tava&dl=1&f=' + tava_laz + '&no_cache=6391a110f2c5b&page_id=614'
    URL_METS = 'https://geoportaal.maaamet.ee/index.php?lang_id=1&plugin_act=otsing&kaardiruut=' + str(map_sq) + '&andmetyyp=lidar_laz_mets&dl=1&f=' + mets_laz + '&no_cache=6391a110f2bee&page_id=614'
    
    #print('\n\n', URL_TAVA)
    #print('\n', URL_METS)
    
    #print('\n\n', tava_file)
    #print('\n', mets_file)
    
    
    if os.path.isfile(tava_file):
        # check if mets file also exists
        if os.path.isfile(mets_file):
                # if it also exists, skip
                print('files already exist: ', map_sq)
                continue
        else:
            # if mets file doesn't exists, create it:
            try:
                open(mets_file, 'a').close()
            except OSError:
                print('Failed creating the mets file ', map_sq ,' ', iindex)

            try:
                request.urlretrieve(URL_METS, mets_file)
            except error.HTTPError as e:
                # Return code error (e.g. 404, 501, ...)
                print('HTTPError: {}'.format(e.code))
    else:
        # if tava file doesn't exist create it
        try:
            open(tava_file, 'a').close()
        except OSError:
            print('Failed creating the tava file ', map_sq)
        try:
            #conn = request.urlopen(URL_TAVA)
            request.urlretrieve(URL_TAVA, tava_file)
            
        except error.HTTPError as e:
            # Return code error (e.g. 404, 501, ...)
            # ...
            print('HTTPError: {}'.format(e.code))
        
        
        # also check here if mets file already exists
        if os.path.isfile(mets_file):
                # if it also exists, skip
                continue
        else:
            # if mets file doesn't exists, create it:
            try:
                open(mets_file, 'a').close()
            except OSError:
                print('Failed creating the mets file ', map_sq)
            #else:
                #print(map_sq, ' mets created')
                # Download the laz file
            try:
                request.urlretrieve(URL_METS, mets_file)
            except error.HTTPError as e:
                # Return code error (e.g. 404, 501, ...)
                print('HTTPError: {}'.format(e.code))
    

    
    

files already exist:  377651
files already exist:  377652
files already exist:  377655
files already exist:  378651
